In [1]:
import json
import random
import numpy as np
import pandas as pd

import utils.editor as edit

from utils.plotter import plot_gantt_jobs
from utils.plotter import plot_gantt_machines

from utils.checker import check_all_constraints

from utils.scheduler import generate_job_arrivals
from utils.scheduler import solve_jobshop_optimal
from utils.scheduler import solve_stage2_early_starts


def print_jobs_compact(job_dict):
    for job, tasks in job_dict.items():
        print(f"{job}:\t{tasks}")

# Matrix-List -----------------------------------------------
def add_job_matrix_to_list(job_set_list, new_instance):
    matrix_index = len(job_set_list)
    job_set_list.append({
        f"Job {matrix_index:02d}_{i}": ops
        for i, ops in enumerate(new_instance.values())
    })

def create_job_matrix_variants_list(instance, num_variants=5):
    job_set_list = []
    for _ in range(num_variants):
        add_job_matrix_to_list(job_set_list, instance)
    return job_set_list


# Datei laden
with open("data/jobshop_instances_named.json", "r", encoding="utf-8") as f:
    jobshop_instances = json.load(f)

instance =  jobshop_instances["instance ft10"]

def plan_day(matrix_index, this_day_jobs, remaining_jobs, solver_time_limit=300, working_day_hours=24):
    print(f"\n==== Tag {matrix_index} ====")

    # Schritt 1: neue Ankunftszeiten erzeugen
    df_arrivals = generate_job_arrivals(this_day_jobs, u_b_mmax=0.9, day_id=matrix_index)

    # Falls noch verbliebene Job-Operationen vom Vortag da sind, hinzufügen
    if remaining_jobs is not None:
        df_arrivals = edit.add_remaining_jobs_with_zero_arrival(df_arrivals, remaining_jobs, day_id=matrix_index)
        this_day_jobs = edit.merge_jobs(remaining_jobs, this_day_jobs)
        print(f"\t{len(remaining_jobs)} Jobs vom Vortag werden berücksichtigt!")
        print_jobs_compact(remaining_jobs)
        print("----------------------------")
    

    # Schritt 2: Erster Jobshop-Solver (Stage 1)
    df_schedule_highs, opt_makespan = solve_jobshop_optimal(
        this_day_jobs, df_arrivals, day_id=matrix_index, solver_time_limit=solver_time_limit
    )

    # Schritt 3: Zweiter Solver für frühe Starts (Stage 2)
    df_schedule_early_starts, final_makespan = solve_stage2_early_starts(
        this_day_jobs, df_arrivals, opt_makespan, day_id=matrix_index, solver_time_limit=solver_time_limit
    )
    print(f"\tFinaler Makespan: {final_makespan} Minuten")

    # Überprüfen
    check_all_constraints(df_schedule_early_starts, this_day_jobs, df_arrivals)

    # Schritt 4: Zeitplan aufteilen
    df_schedule_on_time, df_late = edit.separate_operation_by_day_limit(df_schedule_early_starts, day_limit_h=working_day_hours)

    # Schritt 5: Übrig gebliebene Jobs extrahieren
    remaining_jobs = edit.get_jssp_from_schedule(df_late) if not df_late.empty else None

    if remaining_jobs is not None:
        print(f"\tAnzahl verbliebener Operationen für nächsten Tag: {len(remaining_jobs)}")
    else:
        print(f"\tAlle Jobs abgeschlossen, keine Überträge auf den nächsten Tag.")

    # Nur zurückgeben, nicht zusammenbauen!
    return remaining_jobs, df_schedule_on_time

In [2]:
import ProductionDaySimulation as daysim

In [3]:
def plan_days_in_horizon(current_day, planning_horizon, job_set_list, remaining_jobs, remaining_jobs_current_day,
                         max_solver_time, working_day_hours=24):
    df_schedule_daily = pd.DataFrame()

    for day_id in range(current_day, current_day + planning_horizon):
        if day_id >= len(job_set_list):
            print(f" Tag {day_id} überschreitet die verfügbare Instanzliste. Abbruch.")
            break

        this_day_jobs = job_set_list[day_id]

        remaining_jobs, df_schedule_planned_on_time = plan_day(
            day_id, this_day_jobs, remaining_jobs, max_solver_time, working_day_hours
        )
        df_schedule_daily = pd.concat([df_schedule_daily, df_schedule_planned_on_time], ignore_index=True)

        if day_id == current_day:
            remaining_jobs_current_day = remaining_jobs

    # --- Simulation NACH der Planung ---
    df_schedule_current_day = df_schedule_daily[df_schedule_daily["Day-ID"] == current_day]

    if not df_schedule_current_day.empty:
        print(f"\n--- Simulation für Simulationstag {current_day} ---\n")
        simulation = daysim.ProductionDaySimulation(df_schedule_current_day, vc=0.25)
        df_execution, df_undone = simulation.run(until=1440)
        sim_undone_jobs = edit.get_jssp_from_schedule(df_undone, duration_column="Planned Duration")
        
        remaining_jobs_current_day = edit.merge_jobs(remaining_jobs_current_day, sim_undone_jobs)

    return df_schedule_daily, remaining_jobs_current_day


def run_simulation(df_schedule_daily_list, job_set_list, current_day, remaining_jobs_current_day, total_simulation_days,
                   max_planning_horizon, max_solver_time, working_day_hours=24):
    while current_day < total_simulation_days:
        remaining_jobs = remaining_jobs_current_day
        print(f"\n####### Planung ab Simulationstag {current_day} #######")

        planning_horizon = min(max_planning_horizon, total_simulation_days - current_day)
        print(
            f" Planungshorizont: {planning_horizon} Tage (von Tag {current_day} bis {current_day + planning_horizon - 1})")

        df_schedule_daily, remaining_jobs_current_day = plan_days_in_horizon(
            current_day, planning_horizon, job_set_list, remaining_jobs,
            remaining_jobs_current_day, max_solver_time, working_day_hours
        )

        df_schedule_daily_list.append(df_schedule_daily)

        current_day += 1

    return df_schedule_daily_list

In [4]:
# --- Parameter setzen ---
total_simulation_days = 3
max_planning_horizon = 2
max_solver_time = 110  # Sekunden
working_day_hours = 24

# --- Setup ---
df_schedule_daily_list = []
job_set_list = create_job_matrix_variants_list(instance, total_simulation_days)
current_day = 0
remaining_jobs_current_day = None

# --- Simulation starten ---
df_schedule_daily_list = run_simulation(
    df_schedule_daily_list,
    job_set_list,
    current_day,
    remaining_jobs_current_day,
    total_simulation_days,
    max_planning_horizon,
    max_solver_time,
    working_day_hours
)


####### Planung ab Simulationstag 0 #######
 Planungshorizont: 2 Tage (von Tag 0 bis 1)

==== Tag 0 ====
	Finaler Makespan: 1346.37 Minuten
	✅ Alle Constraints wurden erfüllt.

	Alle Jobs abgeschlossen, keine Überträge auf den nächsten Tag.

==== Tag 1 ====
	Finaler Makespan: 1346.37 Minuten
	✅ Alle Constraints wurden erfüllt.

	Alle Jobs abgeschlossen, keine Überträge auf den nächsten Tag.

--- Simulation für Simulationstag 0 ---

[83.6] Job 00_5 started on M2
[107.2] Job 00_8 started on M0
[171.0] Job 00_5 finished on M2 after 87.45 minutes
[171.0] Job 00_7 started on M2
[189.8] Job 00_8 finished on M0 after 82.59 minutes
[189.8] Job 00_8 started on M1
[189.8] Job 00_0 started on M0
[192.3] Job 00_7 finished on M2 after 21.25 minutes
[221.5] Job 00_0 finished on M0 after 31.65 minutes
[221.5] Job 00_7 started on M0
[244.3] Job 00_8 finished on M1 after 54.52 minutes
[252.2] Job 00_8 started on M3
[252.3] Job 00_5 started on M1
[254.6] Job 00_5 finished on M1 after 2.27 minutes
[254.